![LOGO](../../../img/MODIN_ver2_hrz.png)

<center><h2>Scale your pandas workflows by changing one line of code</h2>


# Exercise 3: Not Implemented

**GOAL**: Learn what happens when a function is not yet supported in Modin as well as how to extend Modin's functionality using the DataFrame Algebra.

When functionality has not yet been implemented, we default to pandas

![](../../../img/omnisci_convert_to_pandas.png)

We convert a Modin dataframe to pandas to do the operation, then convert it back once it is finished. These operations will have a high overhead due to the communication involved and will take longer than pandas.

When this is happening, a warning will be given to the user to inform them that this operation will take longer than usual. For example, `DataFrame.mask` is not yet implemented. In this case, when a user tries to use it, they will see this warning:

```
UserWarning: `DataFrame.mask` defaulting to pandas implementation.
```

## Concept for exercise: Default to pandas

In this section of the exercise we will see first-hand how the runtime is affected by operations that are not implemented.

In [ ]:
import modin.pandas as pd
import pandas
import numpy as np
import time
import modin.config as cfg
cfg.StorageFormat.put("omnisci")

frame_data = np.random.randint(0, 100, size=(2**18, 2**8))
df = pd.DataFrame(frame_data)

In [ ]:
pandas_df = pandas.DataFrame(frame_data)

In [ ]:
modin_start = time.time()

print(df.apply(sum))

modin_end = time.time()
print("Modin mask took {} seconds.".format(round(modin_end - modin_start, 4)))

In [ ]:
pandas_start = time.time()

print(pandas_df.apply(sum))

pandas_end = time.time()
print("pandas mask took {} seconds.".format(round(pandas_end - pandas_start, 4)))

## Consider opening a pull request: https://github.com/modin-project/modin/pulls

For a complete list of what is implemented, see the [documentation](https://modin.readthedocs.io/en/latest/UsingPandasonRay/dataframe_supported.html)

### Now that you are able to create custom functions, you know enough to contribute to Modin!